In [85]:
import numpy as np
import csv
import os
import viterbi
import viterbi_no_log
import baseline

### Inizializzazione delle variabili

In [86]:
# Variabili per memorizzare i dati
tags = []
words = []
#array_dev_set = []
emission_P = []
transition_P = []

Una funzione per la lettura dei file .csv con le probabilità

In [87]:
def readProb(reader):
    for row in reader:
        if row:
            if row[0] == 'tags':
                tags = next(reader)
            elif row[0] == 'words':
                words = next(reader)
            #elif row[0] == 'array_dev_set':
             #   array_dev_set = next(reader)    
            elif row[0] == 'emissione':
                section = 'emissione'
                emission_P = []
            elif row[0] == 'transizione':
                section = 'transizione'
                transition_P = []
            elif section == 'emissione':
                emission_P.append([float(x) for x in row])
            elif section == 'transizione':
                transition_P.append([float(x) for x in row])
    # conversione delle matrici python in matrici numpy
    transition_P = np.array(transition_P)
    emission_P = np.array(emission_P)
    
    return tags, words, transition_P, emission_P

### Lettura dei file .csv
Una versione per ogni dataset wikineural

wikineural_en

In [88]:
current_dir = os.getcwd()

# LETTURA FILE PROBABILITA'
file_path = os.path.join(current_dir, 'wikineural_en', 'probabilities.csv')
with open(file_path, 'r', encoding='utf-8') as prob:
    reader = csv.reader(prob)
    section = None
    tags, words, transition_P, emission_P = readProb(reader)

# LETTURA FILE DI TEST
file_path = os.path.join(current_dir, 'wikineural_en', 'test.conllu')
with open(file_path, 'r', encoding='utf-8') as test:
   righe = test.readlines()[:1000]

wikineural_es

wikineural_it

## Decoding
eseguibile con Viterbi normale, con logaritmi o con la baseline semplice

In [89]:
current_sentence = []
current_tags = []
true_tags = []
predicted_tags = []
 

# un ciclo for generale legge tutte le frasi del file test
for riga in righe:
    riga = riga.strip()
    if riga:
        riga = riga.split()
        current_sentence.append(riga[1])
        current_tags.append(riga[2])
    else:     # riga vuota => end of sentence
        #final_sequence = viterbi_no_log.viterbi_no_log(current_sentence, emission_P, transition_P, tags, words)
        final_sequence = viterbi.viterbi(current_sentence, emission_P, transition_P, tags, words)
        #final_sequence = baseline.easy_baseline(current_sentence, emission_P, tags, words)
        print("frase: ")
        print(current_sentence)
        print("tags: ")
        print(final_sequence)
        # aggiorna la sequenza di frasi e tag corretti per la valutazione
        true_tags.append(current_tags)
        predicted_tags.append(final_sequence)
        current_sentence = []
        current_tags = []
        final_sequence = []

frase: 
['On', 'this', 'occasion', 'he', 'failed', 'to', 'gain', 'the', 'support', 'of', 'the', 'South', 'Wales', 'Miners', "'", 'Federation', 'and', 'had', 'to', 'stand', 'down', '.']
tags: 
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O']
frase: 
['On', 'both', 'these', 'occasions', 'he', 'was', 'backed', 'by', 'the', 'South', 'Wales', 'Miners', "'", 'Federation', ',', 'but', 'he', 'was', 'not', 'successful', '.']
tags: 
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
frase: 
['He', 'also', 'appeared', 'as', 'himself', 'in', 'the', '1996', 'film', '"', 'Eddie', '"', '.']
tags: 
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'O', 'O']
frase: 
['The', 'Colorado', 'Rockies', 'were', 'created', 'as', 'an', 'expansion', 'franchise', 'in', '1993', 'and', 'Coors', 'Field', 'opened', 'in', '1995', '.']
tags: 
['O', 'B-ORG

## Valutazione
Accuracy generale + Precision e Recall sulle entità

In [90]:
# accuracy generale (how many tag are correct?)
# precision e recall sulle entità


# true_tags e predicted_tags sono liste di liste 

correct = 0 
total = 0
predicted_entity = []
true_entity = []
true_positives = 0 
false_positives = 0 
false_negatives = 0
    
for sequence_index, tag_sequence in enumerate(true_tags):  # enumerate() restituisce sia indice che valore
    total += len(tag_sequence)
    for tag_index, true_tag in enumerate(tag_sequence):      
        predicted_tag = predicted_tags[sequence_index][tag_index]
        if true_tag == predicted_tag:  #per Accuracy
            correct += 1
        if predicted_tag.startswith("B-") or predicted_tag.startswith("I-"):
            predicted_entity.append(predicted_tag)    # crea due sequenze di entità, predetta e vera 
            true_entity.append(true_tag)
        if predicted_tag == "O":
            if predicted_entity:    # se abbiamo appena superato un'entità controlliamo se è corretta
                if predicted_entity == true_entity: 
                    true_positives += 1
                else: 
                   false_positives += 1  
                predicted_entity = []
                true_entity = []        
        if true_tag.startswith("B-") and true_tag != predicted_tag:  # se c'è entità non riconosciuta
            false_negatives += 1

accuracy = correct / total 
precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0 
recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0 
 
 
print(f"Accuracy: {accuracy:.4f}") 
print(f"Precision: {precision:.4f}") 
print(f"Recall: {recall:.4f}")


Accuracy: 0.9468
Precision: 0.7962
Recall: 0.7394
